In [1]:
import requests

api_key = "20211595:29e2994c-2337-48d4-a828-4a8f424edfd0"
endpoint = "https://iaapi.novaims.unl.pt/v1/completions"
def llama_completion(prompt, max_tokens=1024):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "Gemma3:12b",
        "prompt": prompt,
        "max_tokens": max_tokens
    }
    response = requests.post(endpoint, headers=headers, json=data)
    return response.json()

def main():
    prompt = "Conta uma anedota sobre programadores"
    response = llama_completion(prompt)
    print(response["choices"][0]["text"])

if __name__ == "__main__":
    main()

Claro, aqui vai uma anedota sobre programadores que sempre me faz rir:

Dois programadores estão trabalhando em um projeto complexo. Um deles, o João, está com um problema particularmente difícil e está falando sozinho, frustrado:

"Por que isso não funciona? Eu declarei a variável, defini o tipo, fiz a chamada de função... Não pode ser!"

O outro programador, o Pedro, que estava do outro lado da sala, levantou a cabeça e perguntou:

"João, você está falando com o código de novo?"

João respondeu, sem tirar os olhos da tela: "Não, eu estou dialogando com ele. Ele me deve uma explicação."

Pedro balançou a cabeça sorrindo e disse: "Acho que você precisa de um café."

A anedota ilustra o quão profundo o relacionamento entre um programador e o código que ele escreve pode ser. É quase como se estivessem conversando diretamente com a máquina! 😂

Espero que tenha gostado!


**ESSENCIAL**

In [2]:
import os
from openai import AzureOpenAI
import api

endpoint = "https://ai-bcds.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = api.key
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


**CHATBOT BASE sem ler documentos**

In [3]:
# === CHAT FUNCTION ===
def ask_financial_bot(user_input: str) -> str:
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful and professional financial assistant. "
                "Always give clear, trustworthy financial advice, and explain things in a beginner-friendly way."
            )
        },
        {
            "role": "user",
            "content": user_input
        }
        
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024,
        temperature=0.7,
        top_p=1.0
    )

    return response.choices[0].message.content

# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_financial_bot(question)
    print("\n💬 Answer:", answer)


Ask your financial assistant:  bla



💬 Answer: It looks like your message got cut off or might not have come through as intended. How can I assist you with your financial questions or concerns today?


**CHATBOT a ler 1 documento**

In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

pdf_text = extract_text_from_pdf("docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf")

def ask_with_pdf_context(user_input):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a financial assistant. Use the following document as context:\n\n"
                + pdf_text[:5000]  # Keep it short to avoid hitting token limits
            )
        },
        {"role": "user", "content": user_input}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


In [5]:
# === EXAMPLE USE ===
if __name__ == "__main__":
    question = input("Ask your financial assistant: ")
    answer = ask_with_pdf_context(question)
    print("\n💬 Answer:", answer)


Ask your financial assistant:  Quais os riscos associados ao Fidelidade Savings para o cliente?



💬 Answer: Os riscos associados ao Fidelidade Savings para o cliente dependem da opção escolhida:

1. **Opção Seguro**: Esta opção tem capital e rendimento garantido, portanto, os riscos são minimizados.

2. **Opção Proteção**: Esta opção garante 90% do capital investido na data de vencimento, mas ainda expõe o cliente a riscos de rendimento e capital.

3. **Opção Dinâmico**: Aqui, há 100% de exposição ao Índice Multiativos, o que implica um maior risco de rendimento e capital.

4. **Opção ESG**: Esta opção também envolve risco, pois investe no índice Multiativos ESG, que considera preocupações ambientais, sociais e de governança.

Em resumo, a escolha da opção influencia diretamente o nível de risco que o cliente está disposto a aceitar.


**CHATBOT a ler vários documentos**

In [8]:
import fitz

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join(page.get_text() for page in doc)

In [9]:
import os

pdf_dir = "docs/Documents for training and evaluation-20250507"
all_texts = []

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(path)
        all_texts.append(text)


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents(all_texts)
texts = [chunk.page_content for chunk in chunks]


In [11]:
from openai import AzureOpenAI
import numpy as np

embedding_client = AzureOpenAI(
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-02-01",
    azure_endpoint="https://ai-bcds.openai.azure.com/"
)

def get_embedding(text):
    response = embedding_client.embeddings.create(
        input=[text],
        model="text-embedding-3-small"  # Use the deployment name you set up
    )
    return np.array(response.data[0].embedding).astype('float32')

In [12]:
#%pip install faiss-cpu
import faiss

In [13]:
embeddings = np.array([get_embedding(text) for text in texts])

# Check the shape of embeddings (Should be (N, D) where N is number of chunks and D is embedding dimension)
print(embeddings.shape)

# Use IndexFlatL2 (no training needed)
d = 1536  # Dimensionality of the embeddings (1536 for text-embedding-3-small)
index = faiss.IndexFlatL2(d)  # This index does not require training

# Add embeddings to the index
index.add(embeddings)


(18, 1536)


In [14]:
def get_top_k_chunks(query, k=3):
    query_vec = np.array([get_embedding(query)]).astype('float32')
    D, I = index.search(query_vec, k)  # Get top-k similar chunks
    return [texts[i] for i in I[0]]  # Return the corresponding text chunks

In [ ]:
# Example usage
query = "What are the latest trends in financial advising?"
top_chunks = get_top_k_chunks(query)
print(top_chunks)

In [28]:
%pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.
